In [9]:
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import pinecone
from langchain_community.vectorstores import Pinecone
from langchain.chains import RetrievalQA


/opt/anaconda3/envs/langchain-apps/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [10]:
db_schema = open('db_schema.txt', 'r')
tokens = 0
for line in db_schema:
    tokens += len(line.split())
print(f'Número de tokens: {tokens}')

Número de tokens: 7535


In [11]:

splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    length_function=len
)

In [12]:
with open('db_schema.txt') as f:
   db_schema = f.read() 
chunks = splitter.create_documents([db_schema])

In [13]:

embeddings = OpenAIEmbeddings()
set_llm_cache(SQLiteCache(database_path='./cache/.langchain.db'))

/opt/anaconda3/envs/langchain-apps/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
def print_embedding_cost(text):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in text])
    print(f"Total tokens: {total_tokens}")
    print('Embedding cost in USD:',total_tokens/1000 * 0.0004)

In [15]:
print_embedding_cost(chunks)

Total tokens: 40851
Embedding cost in USD: 0.0163404


In [18]:
pc = pinecone.Pinecone()

for i in pc.list_indexes().names():
    print('Deletando indice ', i)
    pc.delete_index(i)

Deletando indice  geoex-sql-embeddings


In [172]:
index_name = 'geoex-sql-embeddings'
if index_name not in pc.list_indexes().names():
    print('Criando index',index_name)
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=pinecone.PodSpec(
            environment='gcp-starter'
        )
        
    )
    print('done')

Criando index geoex-sql-embeddings
done


Criando VectorStore dos metadados do Banco

In [173]:
vector_store = Pinecone.from_documents(chunks,embeddings,index_name=index_name)

In [174]:
pc.Index(index_name).describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00652,
 'namespaces': {'': {'vector_count': 652}},
 'total_vector_count': 652}

In [175]:
query = 'Projeto Normal'
result = vector_store.similarity_search(query)
print(result)

[Document(page_content='ProjetoPEPId'), Document(page_content='PlanejamentoTipoObraId CustoEstimado'), Document(page_content='ProjetoEstudoAmbientalAcaoVegetacaoIdParente'), Document(page_content='ProjetoEstudoAmbientalAcaoTipoId ProjetoEstudoAmbientalAcaoHistoricoId')]


In [176]:


llm = ChatOpenAI(model='gpt-4',temperature=0)

In [177]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':50})
chain = RetrievalQA.from_chain_type(llm=llm,chain_type='stuff',retriever=retriever)

In [178]:
prompt_template = """Voce é um assistente chamado GeoAI que so responde em formato de script de
    consulta em SQL para SQL server, voce so faz comandos de SELECT e Nunca de Alteração dos dados ou metadados. Baseado nos dados fornecidos, monte a melhor consulta 
    para responder esta pergunta de forma mais ao pé da letra possível: '{question}'
    """


# Pergunta do usuário 
question = "Me de um relatorio das medicoes fechadas de forma parcial"
prompt_template = PromptTemplate.from_template(template=prompt_template)
prompt = prompt_template.format(question=question)

In [180]:
# Output modelo
resp = chain.run(prompt)
print(resp)

SELECT * FROM MedicaoFechamento WHERE Parcial = 1
